# Django de un vistazo¶

Debido a que Django fue desarrollado en un entorno de sala de redacción de ritmo rápido, fue diseñado para hacer que las tareas comunes de desarrollo web sean rápidas y fáciles. Aquí hay una descripción informal de cómo escribir una aplicación web impulsada por una base de datos con Django.

El objetivo de este documento es darle suficiente información técnica para entender cómo funciona Django, pero no pretende ser un tutorial o referencia - ¡Aunque tenemos ambas cosas! Cuando esté listo para empezar un proyecto con Django, puede comenzar con el tutorial https://docs.djangoproject.com/es/5.0/intro/tutorial01/ o ir directamente a la documentación más detallada.

## Diseñe su modelo

Aunque es posible usar Django sin una base de datos, Django cuenta con un mapeador objeto-relacional en el que es posible definir la estructura de la base de datos utilizando código Python.

La sintaxis de modelo de datos ofrece muchas formas de representar sus modelos – hasta ahora, ha resuelto una cantidad equivalente a años de problemas de esquemas de base de datos. Aquí hay un ejemplo rápido:

In [ ]:
# project/app/models.py

from django.db import models


class Reporter(models.Model):
    full_name = models.CharField(max_length=70)

    def __str__(self):
        return self.full_name


class Article(models.Model):
    pub_date = models.DateField()
    headline = models.CharField(max_length=200)
    content = models.TextField()
    reporter = models.ForeignKey(Reporter, on_delete=models.CASCADE)

    def __str__(self):
        return self.headline



### Instálelo

A continuación, ejecute las utilidades de línea de comandos de Django para crear automáticamente las tablas de la base de datos.

$ python manage.py makemigrations

$ python manage.py migrate

El comando makemigrations examina todos los modelos disponibles y crea migraciones para las tablas que aún no existen. migrate ejecuta las migraciones y crea tablas en su base de datos, además de proporcionar opcionalmente un control de esquema mucho más completo.

### Una interfaz administrativa dinámica: no es sólo andamios; es toda una casa¶

Una vez que sus modelos están definidos, Django puede crear automáticamente una interfaz administrativa profesional y lista para producción: un sitio web que permite a los usuarios autenticados agregar, cambiar y eliminar objetos. El único paso requerido es registrar su modelo en el sitio de administración:

In [ ]:
from django.db import models


class Article(models.Model):
    pub_date = models.DateField()
    headline = models.CharField(max_length=200)
    content = models.TextField()
    reporter = models.ForeignKey(Reporter, on_delete=models.CASCADE)

In [ ]:
from django.contrib import admin

from . import models

admin.site.register(models.Article)

La filosofía aquí es que su sitio es editado por un personal, o un cliente, o tal vez solo usted, y no quiere tener que lidiar con la creación de interfaces de backend solo para administrar el contenido.

Un flujo de trabajo típico en la creación de aplicaciones Django es crear modelos y poner a funcionar los sitios administrativos lo más rápido posible para que su personal (o clientes) puedan empezar a ingresar los datos y luego, perfeccionar la forma como los datos se presentan al público.
## Diseñe sus URLs

Un esquema de URL limpio y elegante es un detalle importante en una aplicación web de alta calidad. Django fomenta un diseño de URL hermoso y no añade ningún elemento innecesario en las URLs, como .php o .asp.

Para diseñar las URLs de una aplicación, debe crear un módulo de Python llamado URLconf. Esta es una tabla de contenidos para su aplicación con un un mapeo sencillo entre patrones URLs y funciones callback de Python. Las tablas URLconf también sirven para separar las URLs del código Python.

Así es como se vería una URLconf para el ejemplo Reporter/Article anterior:

In [ ]:
# priject/app/urls.py¶

from django.urls import path

from . import views

urlpatterns = [
    path("articles/<int:year>/", views.year_archive),
    path("articles/<int:year>/<int:month>/", views.month_archive),
    path("articles/<int:year>/<int:month>/<int:pk>/", views.article_detail),
]



El código anterior mapea las URLs a funciones callback de Python («vistas»). Las cadena de caracteres de la ruta usa etiquetas para «capturar» los valores de las URLs. Cuando un usuario solicita una página Django pasa por cada patrón en orden y se detiene en el primero que coincide con la URL solicitada. (Si ninguno de ellos coincide Django llama una vista del caso especial 404). Esto es increíblemente rápido debido a que las rutas son compiladas como expresiones regulares en tiempo de carga.

Una vez que una de las expresiones regulares coincide, Django llama a la vista determinada, que es una función Python. Cada vista recibe un objeto de consulta – que contiene metadatos de consulta – y los valores capturados en la expresión regular.

Por ejemplo si un usuario solicitara la dirección URL «/articles/2005/05/39323/» Django llamaría la función app.views.article_detail(request, year=2005, month=05, pk=39323).
## Escriba sus vistas

Cada vista es responsable de hacer una de dos cosas: retornar un objeto HttpResponse que incluye el contenido de la página solicitada, o levantar una excepción como Http404. El resto depende de usted.

Generalmente, una vista recupera los datos de acuerdo con los parámetros, carga una plantilla y crea la plantilla con los datos recuperados. Aquí está una vista de ejemplo para el year_archive de arriba:

In [ ]:
# project/app/views.py¶

from django.shortcuts import render

from .models import Article


def year_archive(request, year):
    a_list = Article.objects.filter(pub_date__year=year)
    context = {"year": year, "article_list": a_list}
    return render(request, "news/year_archive.html", context)



Este ejemplo utiliza el sistema de plantillas de Django que tiene varias características potentes, pero se esfuerza por mantenerse lo suficientemente simple como para que los que no son programadores lo puedan utilizar.
## Diseñe sus plantillas

El código anterior carga la plantilla year_archive.html.

Django tiene una ruta de búsqueda de plantillas lo que le permite a usted minimizar la redundancia entre las plantillas. En su configuración de Django, usted especifica una lista de directorios para verificar si hay plantillas con DIRS. Si una plantilla no existe en el primer directorio, verifica en el segundo y así sucesivamente.

Digamos que se encontró la plantilla year_archive.html. Así es como podría verse:

In [ ]:
# project/app/templates/year_archive.html

{% extends "base.html" %}

{% block title %}Articles for {{ year }}{% endblock %}

{% block content %}
<h1>Articles for {{ year }}</h1>

{% for article in article_list %}
    <p>{{ article.headline }}</p>
    <p>By {{ article.reporter.full_name }}</p>
    <p>Published {{ article.pub_date|date:"F j, Y" }}</p>
{% endfor %}
{% endblock %}



Las variables están rodeadas por llaves dobles. {{ article.headline }} significa «Enviar el valor del atributo headline del artículo». Sin embargo, los puntos no sólo se utilizan para las operaciones de búsqueda de atributo. También pueden hacer búsqueda de pares diccionario-clave, búsqueda de índice y llamadas a funciones.

Tenga en cuenta que {{ article.pub_date|date:"F j, Y" }} utiliza una «tubería» al estilo Unix (el carácter «|»). Esto se conoce como filtro de plantilla, y es una manera de filtrar el valor de una variable. En este caso, el filtro de fecha formatea un objeto datetime de Python en el formato determinado (como se encuentra en la función de fecha de PHP).

Usted puede encadenar tantos filtros como desee. Usted puede escribir filtros de plantillas personalizados https://docs.djangoproject.com/es/5.0/howto/custom-template-tags/#howto-writing-custom-template-filters. Usted puede escribir etiquetas de plantillas personalizadas https://docs.djangoproject.com/es/5.0/howto/custom-template-tags/ que ejecutan el código Python personalizado tras bastidores.

Por último, Django usa el concepto de «herencia de plantillas». Eso es lo que hace el {% extends "base.html"%}. Esto quiere decir «Primero cargue la plantilla llamada “base”, que ha definido un grupo de bloques, y llene los bloques con los siguientes bloques.» En resumen, eso le permite reducir drásticamente la redundancia en las plantillas: cada plantilla tiene que definir sólo lo que es exclusivo de esa plantilla.

Aquí esta como la plantilla «base.html», incluyendo el uso de static files https://docs.djangoproject.com/es/5.0/howto/static-files/, podría verse:

In [ ]:
# BASE_DIR/templates/base.html

{% load static %}
<html>
<head>
    <title>{% block title %}{% endblock %}</title>
</head>
<body>
    <img src="{% static 'images/sitelogo.png' %}" alt="Logo">
    {% block content %}{% endblock %}
</body>
</html>



De forma simple, define el aspecto del sitio (con el logo del sitio) y proporciona «agujeros» a ser llenados por las plantillas hijas. Esto hace que el rediseño del sitio resulte tan fácil como cambiar un solo archivo: la plantilla base.

También le permite crear múltiples versiones de un sitio, con diferentes plantillas bases, a la vez que reutiliza las plantillas hijas. Los creadores de Django han usado esta técnica para crear versiones móviles de sitios sorprendentemente distintas, simplemente creando una nueva plantilla base.

Tenga en cuenta que usted no tiene que usar el sistema de plantillas de Django si prefiere otro sistema. Si bien el sistema de plantillas de Django está particularmente bien integrado con la capa modelo de Django, nada lo obliga a usarlo. No tiene tampoco que usar la API de base de datos de Django. Puede usar otra capa de abstracción de base de datos, leer archivos XML, leer archivos desde el disco o lo que desee. Cada parte de Django – modelos, vistas, plantillas – está separada de la otra.

## Esto es sólo la superficie¶

Esta ha sido solamente una rápida introducción a la funcionalidad de Django. Otras características útiles:

    - Un framework de caché que se integra con memcached u otros backends https://docs.djangoproject.com/es/5.0/topics/cache/.
    
    - Un framework de sindicación que hace que la creación de feeds RSS y Atom sea tan fácil como escribir una pequeña clase de Python https://docs.djangoproject.com/es/5.0/ref/contrib/syndication/.
    
    - Funciones administrativas del sitio, más atractivas y generadas automáticamente – esta descripción general apenas toca la superficie.